In [ ]:
!pip install -q gwpy
# use %%capture

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%%capture
!apt install python3-libtorrent
import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)
#downloads = []

import time
from IPython.display import display
import ipywidgets as widgets


class Downloader:
  def __init__(self, path="/content/drive/My Drive/Movies"):
    params = {"save_path": path}
    self.downloads = []
    while True:
      magnet_link = input("Enter Magnet Link Or Type Exit: ")
      if magnet_link.lower() == "":
          break
      self.downloads.append(
          lt.add_magnet_uri(ses, magnet_link, params)
      )

  def download(self):
    downloads = self.downloads
    state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]

    layout = widgets.Layout(width="auto")
    style = {"description_width": "initial"}
    download_bars = [
        widgets.FloatSlider(
            step=0.01, disabled=True, layout=layout, style=style
        )
        for _ in downloads
    ]
    display(*download_bars)

    while downloads:
        next_shift = 0
        for index, download in enumerate(downloads[:]):
            bar = download_bars[index + next_shift]
            if not download.is_seed():
                s = download.status()

                bar.description = " ".join(
                    [
                        download.name(),
                        str(s.download_rate / 1000),
                        "kB/s",
                        state_str[s.state],
                    ]
                )
                bar.value = s.progress * 100
            else:
                next_shift -= 1
                ses.remove_torrent(download)
                downloads.remove(download)
                bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                download_bars.remove(bar)
                print(download.name(), "complete")
        time.sleep(1)
#temp = Downloader()

In [ ]:
### Downloading ###
d = Downloader(path = "/content")
d.download()

Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:975C64C314B3B7584D9803B1612F16B9E396E476&dn=Stree+2+%282024%29+Hindi+720p+WEBRip+x264+AAC+ESub&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.tracker.cl%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.auctor.tv%3A6969%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.demonii.com%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.moeking.me%3A6969%2Fannounce&tr=udp%3A%2F%2Fexplodie.org%3A6969%2Fannounce&tr=https%3A%2F%2Ftracker.tamersunion.org%3A443%2Fannounce&tr=https%3A%2F%2Ftracker.gbitt.info%3A443%2Fannounce&tr=http%3A%2F%2Ftracker.gbitt.info%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker-udp.gbitt.info%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.tiny-vps.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%

<ipython-input-9-bdcb55ccd3d2>:21: DeprecationWarning: add_magnet_uri() is deprecated
  lt.add_magnet_uri(ses, magnet_link, params)


Enter Magnet Link Or Type Exit: 


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

<ipython-input-9-bdcb55ccd3d2>:51: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
<ipython-input-9-bdcb55ccd3d2>:56: DeprecationWarning: name() is deprecated
  download.name(),
<ipython-input-9-bdcb55ccd3d2>:69: DeprecationWarning: name() is deprecated
  print(download.name(), "complete")


Stree 2 (2024) Hindi 720p WEBRip x264 AAC ESub.mkv complete


# Copying to Drive

In [ ]:
# # mount it
# from google.colab import drive
# drive.mount('/content/drive')
# # copy it there
# !cp /content/'Stree 2 (2024) Hindi 720p WEBRip x264 AAC ESub.mkv' /content/drive/MyDrive/Movies/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Add from torrent file

In [ ]:
# # For uploading torrent file from local pc
# from google.colab import files

# source = files.upload()
# params = {
#     "save_path": "/content/drive/My Drive/Torrent",
#     "ti": lt.torrent_info(list(source.keys())[0]),
# }
# downloads.append(ses.add_torrent(params))

## Add from magnet link

In [ ]:
# params = {"save_path": "/content/drive/My Drive/Torrent"}

# while True:
#     magnet_link = input("Enter Magnet Link Or Type Exit: ")
#     if magnet_link.lower() == "exit":
#         break
#     downloads.append(
#         lt.add_magnet_uri(ses, magnet_link, params)
#    )

## Start Download

In [ ]:
# import time
# from IPython.display import display
# import ipywidgets as widgets

# state_str = [
#     "queued",
#     "checking",
#     "downloading metadata",
#     "downloading",
#     "finished",
#     "seeding",
#     "allocating",
#     "checking fastresume",
# ]

# layout = widgets.Layout(width="auto")
# style = {"description_width": "initial"}
# download_bars = [
#     widgets.FloatSlider(
#         step=0.01, disabled=True, layout=layout, style=style
#     )
#     for _ in downloads
# ]
# display(*download_bars)

# while downloads:
#     next_shift = 0
#     for index, download in enumerate(downloads[:]):
#         bar = download_bars[index + next_shift]
#         if not download.is_seed():
#             s = download.status()

#             bar.description = " ".join(
#                 [
#                     download.name(),
#                     str(s.download_rate / 1000),
#                     "kB/s",
#                     state_str[s.state],
#                 ]
#             )
#             bar.value = s.progress * 100
#         else:
#             next_shift -= 1
#             ses.remove_torrent(download)
#             downloads.remove(download)
#             bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
#             download_bars.remove(bar)
#             print(download.name(), "complete")
#     time.sleep(1)
